# Week 02

Content:

1. Language modelling using bigrams
2. Necessity of end of sentence token `<\s>`
3. Doing k-smoothing, we are still dealing with a probability
4. Makemore Part I

### 1. Language modelling using bigrams



In [1]:
import nltk
nltk.download("gutenberg")
emma = nltk.corpus.gutenberg.raw(fileids='austen-emma.txt')

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/gideon/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
print(emma[:1000])

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died too long ago for her to have more than an indistinct
remembrance of her caresses; and her place had been supplied
by an excellent woman as governess, who had fallen little short
of a mother in affection.

Sixteen years had Miss Taylor been in Mr. Woodhouse's family,
less as a governess than a friend, very fond of both daughters,
but particularly of Emma.  Between _them_ it was more the intimacy
of sisters.  Even before Miss Taylor had ceased to hold the nominal
office of governess, the mildness o

Split into sentences:

In [3]:
from nltk.tokenize import WhitespaceTokenizer, word_tokenize, sent_tokenize
sentences = sent_tokenize(emma.lower())
sentences[:5]

['[emma by jane austen 1816]\n\nvolume i\n\nchapter i\n\n\nemma woodhouse, handsome, clever, and rich, with a comfortable home\nand happy disposition, seemed to unite some of the best blessings\nof existence; and had lived nearly twenty-one years in the world\nwith very little to distress or vex her.',
 "she was the youngest of the two daughters of a most affectionate,\nindulgent father; and had, in consequence of her sister's marriage,\nbeen mistress of his house from a very early period.",
 'her mother\nhad died too long ago for her to have more than an indistinct\nremembrance of her caresses; and her place had been supplied\nby an excellent woman as governess, who had fallen little short\nof a mother in affection.',
 "sixteen years had miss taylor been in mr. woodhouse's family,\nless as a governess than a friend, very fond of both daughters,\nbut particularly of emma.",
 'between _them_ it was more the intimacy\nof sisters.']

To Do:
- count all bigrams in sentences above (add a start and end of sentence token), since we want to generate new sentences
- construct probabilities from the bigram counter above
- to generate new sentences you start with `<s>` and generate a new word from the bigram probabilities until you reach the end of sentence token `<\s>`
- finally calculate the perplexity of your generated sentences and do the same for original sentences from `sentences`


Tipps:
- `nltk.utils` has a `ngrams` function
- you might want to use `from collections import Counter`
- `np.random.choice(candidate_tokens, size=1, p=candidate_probs)` can be used to draw tokens given computed probabilities

In [4]:
from nltk.util import ngrams
from collections import Counter
import numpy as np
import math

start_token = "<s>"
end_token = "<\\s>"

tokenized_sentences = []
for sent in sentences:
    tokens = [start_token] + sent.split() + [end_token]
    tokenized_sentences.append(tokens)

bigram_counter = Counter()
for tokens in tokenized_sentences:
    for bigram in list(ngrams(tokens, 2)):
        bigram_counter[bigram] += 1

first_token_counts = Counter()
for (w1, w2), count in bigram_counter.items():
    first_token_counts[w1] += count

bigram_prob = {}
for (w1, w2), count in bigram_counter.items():
    prob = count / first_token_counts[w1]
    bigram_prob[(w1, w2)] = prob

def generate_sentence():
    current_token = start_token
    sentence = []
    while True:
        candidates = []
        candidate_probs = []
        for (w1, w2), prob in bigram_prob.items():
            if w1 == current_token:
                candidates.append(w2)
                candidate_probs.append(prob)
                
        if not candidates:
            break
            
        candidate_probs = np.array(candidate_probs)
        candidate_probs = candidate_probs / candidate_probs.sum()
        next_token = np.random.choice(candidates, size=1, p=candidate_probs)[0]
        
        if next_token == end_token:
            break
        sentence.append(next_token)
        current_token = next_token
        
    return " ".join(sentence)

generated_sentence = generate_sentence()
print("Generierter Satz:", generated_sentence)

def sentence_perplexity(sentence):
    tokens = [start_token] + sentence.split() + [end_token]
    N = len(tokens) - 1 
    log_prob_sum = 0.0
    for i in range(N):
        bigram = (tokens[i], tokens[i+1])
        prob = bigram_prob.get(bigram, 1e-8)
        log_prob_sum += math.log(prob)
        
    perplexity = math.exp(-log_prob_sum / N)
    return perplexity

import pandas as pd

data = []
for idx, sent in enumerate(sentences):
    perplex = sentence_perplexity(sent)
    data.append({"Index": idx + 1, "Satz": sent, "Perplexity": round(perplex, 2)})

gen_perplexity = sentence_perplexity(generated_sentence)
print("\nPerplexity des generierten Satzes:", gen_perplexity)

df = pd.DataFrame(data)
print("\n")
print(df.describe())

Generierter Satz: "that will be applied herself well be a persuasion.

Perplexity des generierten Satzes: 42.989408329366725


             Index   Perplexity
count  7489.000000  7489.000000
mean   3745.000000    37.774621
std    2162.032416    14.338568
min       1.000000     6.340000
25%    1873.000000    29.080000
50%    3745.000000    36.000000
75%    5617.000000    43.710000
max    7489.000000   181.420000


## 2. Necessity of `<\s>`

Exercise 3.5 of [Speech and Language Processing](https://web.stanford.edu/~jurafsky/slp3/). Given a trainings corpus without `<\s>`

`<s> a b` <br>
`<s> b a` <br>
`<s> a a` <br>
`<s> b b` <br>

1. Use a bigram model and calculate the probability of all possible sentences with two words $\{a, b\}$. Show that the sum of all these probabilities add up to 1.
2. Do the same for all possible sentences of lengths 3 of the words $\{a, b\}$. Show that these sum up to 1 as well.

The point is, that this property is not very useful: we rather have a language model, that is able to produce a sentence of arbitrary length. When you are generating a sentence, you are not going to decide before hand, how many words you are going to use...

### Answer

First let's write down the formula for bigrams:

$$
\mathbb P [ <\! s \! >, w_1, \dots, w_n] = \mathbb P [w_1 \mid <\!s \! > ] \mathbb P[w_2 \mid w_1] \cdots \mathbb P[w_n \mid w_{n-1}]
$$

here $n=2$ for part 1 and $n=3$ for part 2.

Let's write $\mathbb P[b \mid a] = p_{b|a}$ and use $s$ instead of `<s>`, then from the training corpus we get:


- $p_{a|s} = p_{b|s} = \dots$
- $\dots$

Recall that we are counting the number of occurrences, e.g.: $p_{b|a}= \frac{C(a,b)}{C(a)} = \frac{1}{2}$

Bigram Wahrscheinlichkeiten:
- $C(a) = 4$
- $C(b) = 4$
 
- $C(s, a) = 2$
- $C(s, b) = 2$
- $C(a, a) = 1$
- $C(a, b) = 1$
- $C(b, a) = 1$
- $C(b, b) = 1$

- $p_{a|s} = p_{b|s} = \frac{C(s, a)}{C(a)} = \frac{2}{4} = \frac{1}{2}$


#### Answer 2

We must calculate all probabilities of sentences of lengths 3:

- How many are there?

...


$|V| = 2$

daraus ergibt sich das es $2^3 = 8$ mögliche Sätze gibt:

## 3. k-smoothing is still a probability

Given a vocabulary $V$ and a bigram language model, applying k-smoothing (or additive smoothing), i.e.

$$\mathbb{P}[w_n \mid w_{n-1}] = \frac{C(w_n, w_{n-1}) + k}{ C(w_{n-1}) + k|V|}$$

we still have a probability.

We have to show, that when summing over all possible $w_n$, the sum still adds up to $1$:

$$
\begin{align*}
    \sum_{w_n \in V} \mathbb P [w_n \mid w_{n-1}] 
        &=\sum_{w_n \in V} \frac{C(w_{n-1}, w_n) + k}{ C(w_{n-1}) + k|V|} \\
        &=  \frac{1}{C(w_{n-1}) + k|V|} \sum_{w_n \in V} (C(w_{n-1}, w_n) + k) \\ 
        &=  \frac{1}{C(w_{n-1}) + k|V|} (\sum_{w_n \in V} C(w_{n-1}, w_n) + \sum_{w_n \in V} k) \\ 
        &=  \frac{1}{C(w_{n-1}) + k|V|} (C(w_{n-1}) + k|V|) \\ 
        &=  \frac{C(w_{n-1}) + k|V|}{C(w_{n-1}) + k|V|} \\ 
        = 1
\end{align*}
$$

Zusätzliche Erklärung:

Die Summe aller auf $w_{n-1}$ folgenden Wörter ergibt grad die Anzahl der vorkommnisse von $w_{n-1}$ also $C(w_{n-1})$.
$\sum_{w_n \in V} C(w_{n-1}, w_n) = C(w_{n-1})$

Die Summe von $k$ über alle Wörter ergibt $k|V|$, da es $|V|$ Wörter gibt.
$ \sum_{w_n \in V} k = k|V|$

# 4. Makemore Part I

Watch Part I of [makemore](https://youtu.be/PaCmpygFfXo?si=7PonsCOBoNCcmWHo), until 1:02:56, and code along. We will come back to this video series in later weeks.

- Write a function, that takes a prefix and completes this prefix according to the the implemented bigram-model (e.g. given the prefix `ann` possible completions could be `anna`, `anner`, `ann...`) 
- Loglikelihood:
    - what happens to the loglikelihood if we use large smoothing factor $k$? 
    - Calculate the loglikelihood for all words for different $k$ values.
    - make an appropriate plot with negative loglikelihood on the y-axis and logarithmic $k$ on the x-axis.